# DNN with TensorFlow

## load dataset

In [7]:
import tensorflow as tf
import numpy as np

load MNIST from local dir

In [8]:
from mlxtend.data import loadlocal_mnist

In [9]:
X_train, Y_train = loadlocal_mnist(images_path=r'/home/bob/scikit_learn_data/train-images.idx3-ubyte', 
                                   labels_path=r'/home/bob/scikit_learn_data/train-labels.idx1-ubyte')

X_test, Y_test = loadlocal_mnist(images_path=r'/home/bob/scikit_learn_data/t10k-images.idx3-ubyte', 
                                 labels_path=r'/home/bob/scikit_learn_data/t10k-labels.idx1-ubyte')

# normalization
X_train = X_train.astype(np.float32) / 255.0
X_test = X_test.astype(np.float32) / 255.0

# validation set
X_validation, X_train = X_train[:5000],X_train[5000:]
Y_validation, Y_train = Y_train[:5000], Y_train[5000:]

In [10]:
print('Dimensions of X_train:{}x{}'.format(X_train.shape[0], X_train.shape[1]))

Dimensions of X_train:55000x784


illustration of y

In [11]:
print('labels_unique : {}'.format(np.unique(Y_train)))

# Count number of occurrences of each value in array of non-negative ints.
print('Class distributions : {}'.format(np.bincount(Y_train)))

labels_unique : [0 1 2 3 4 5 6 7 8 9]
Class distributions : [5444 6179 5470 5638 5307 4987 5417 5715 5389 5454]


##  construct DNN with tf.layers.dense()

specify hyperparameters

In [12]:
n_inputs = 28*28
n_hidden_1 = 150
n_hidden_2 = 150
n_output = 10    # logits

In [13]:
# tf.reset_default_graph()

In [14]:
# image placeholder
image = tf.placeholder(dtype=tf.float32, shape=(None, n_inputs), name='input_layer')
# label placeholder (should be ranked None)
label = tf.placeholder(dtype=tf.int32, shape=(None), name="output_layer")

### DNN unit

In [15]:
with tf.name_scope("DNN"):
    
    """ Output tensor the same shape as `inputs` except the last dimension is of
  size `units`"""
    
    hidden1 = tf.layers.dense(inputs=image, units=n_hidden_1, name="hidden1", 
                              activation=tf.nn.relu)
    hidden2 = tf.layers.dense(inputs=hidden1, units=n_hidden_2, name="hidden2", 
                              activation=tf.nn.relu)
    output_logits = tf.layers.dense(inputs=hidden2, units=n_output, name="output")
    y_prob = tf.nn.softmax(output_logits, axis=1)

### loss unit(cross entropy)

In [16]:
with tf.name_scope("loss"):
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=label, logits=output_logits)
    mean_cross_entropy = tf.reduce_mean(cross_entropy)

### optimization unit

In [17]:
learning_rate = 0.01

In [18]:
with tf.name_scope("optimization"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(mean_cross_entropy)

### evaluation metrics unit (accuracy)

In [19]:
with tf.name_scope("evaluation"):
    # return a tensor of batch-size showing how many of them are predicted correctly (one-hot vector)
    correct_bool = tf.nn.in_top_k(predictions=output_logits, targets=label, k=1)
    accuracy = tf.reduce_mean(tf.cast(correct_bool, tf.float32))

### fetch batch

In [20]:
def fetch_batch(dataset_X, dataset_Y, epoch_index, batch_index, batch_size):
    np.random.seed(epoch_index*batch_size+batch_index)
    selected_samples = np.random.randint(X_train.shape[1], size=batch_size)
    X_batch = dataset_X[selected_samples]
    Y_batch = dataset_Y[selected_samples]
    return X_batch, Y_batch

In [21]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        return X_batch, y_batch

### summary (attach summary node at the end of the graph)

generate logdir

In [27]:
from datetime import datetime
def gen_log():
    now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
    root_dir = "tf_log"
    logdir = "{}/run-{}/".format(root_dir, now)
    print(logdir)
    return logdir

In [28]:
# Writes `Summary` protocol buffers to event files.
loss_summary = tf.summary.scalar(name="summary_cross_entropy", tensor=mean_cross_entropy)
accuracy_summary = tf.summary.scalar(name="summary_accuracy", tensor=accuracy)
file_writer = tf.summary.FileWriter(gen_log(), tf.get_default_graph())

tf_log/run-20190318071355/


In [30]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

## Session

In [25]:
n_epochs = 800
batch_size = 50
n_batches = int(X_train.shape[1] // batch_size)

In [31]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for batch in range(n_batches):
            X_batch, Y_batch = fetch_batch(X_train, Y_train, epoch, batch, batch_size)
#             X_batch, Y_batch = shuffle_batch(X_train, Y_train, batch_size)
            sess.run(training_op, feed_dict={image: X_batch, label: Y_batch})
            step = epoch*n_batches+batch
            step_loss, step_accuracy = sess.run([loss_summary, accuracy_summary], feed_dict={image: X_batch, label: Y_batch})
            
            if step%10 == 0:
                print("step", step, ":", "cross-entropy:", step_loss)
                print("step", step, ":", "accuracy:", step_accuracy)
                # add 'step_loss' summary log to log_file
                file_writer.add_summary(step_loss, step)
                file_writer.add_summary(step_accuracy, step)
                
        test_accuracy = sess.run(accuracy, feed_dict={image: X_validation, label: Y_validation})
        print("\ntest_accuracy:", test_accuracy, "\n")

step 0 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15\xfc\xf1\r@'
step 0 : accuracy: b'\n\x17\n\x10summary_accuracy\x15\xcd\xccL>'
step 10 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15\xe8\xea\r@'
step 10 : accuracy: b'\n\x17\n\x10summary_accuracy\x15\xecQ8>'

test_accuracy: 0.2352 

step 20 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15h\xb0\x08@'
step 20 : accuracy: b'\n\x17\n\x10summary_accuracy\x15{\x14\xae>'

test_accuracy: 0.3048 

step 30 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15J:\x06@'
step 30 : accuracy: b'\n\x17\n\x10summary_accuracy\x15\x8f\xc2u>'
step 40 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15\x9a\xd4\x06@'
step 40 : accuracy: b'\n\x17\n\x10summary_accuracy\x15=\n\xd7>'

test_accuracy: 0.4162 

step 50 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15l\x08\xf6?'
step 50 : accuracy: b'\n\x17\n\x10summary_accuracy\x15\xb8\x1e\x05?'

test_accuracy: 0.5052 

step 60 : cross-entropy: b'\n\x1e\n\x1


test_accuracy: 0.8394 

step 540 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15X\x80\xf1>'
step 540 : accuracy: b'\n\x17\n\x10summary_accuracy\x15\xf6(\\?'
step 550 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15\xee/\x8c>'
step 550 : accuracy: b'\n\x17\n\x10summary_accuracy\x15\xd7\xa3p?'

test_accuracy: 0.8416 

step 560 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15\x1b\xb4\x14?'
step 560 : accuracy: b'\n\x17\n\x10summary_accuracy\x15\x85\xebQ?'

test_accuracy: 0.8384 

step 570 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15\xc6\xb1\x89>'
step 570 : accuracy: b'\n\x17\n\x10summary_accuracy\x15\xd7\xa3p?'
step 580 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15\x86\xc3\xb9>'
step 580 : accuracy: b'\n\x17\n\x10summary_accuracy\x15\x8f\xc2u?'

test_accuracy: 0.8438 

step 590 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15C\xc5\xe1>'
step 590 : accuracy: b'\n\x17\n\x10summary_accuracy\x15fff?'

test_accuracy: 0.8448 


test_accuracy: 0.8672 

step 1080 : cross-entropy: b"\n\x1e\n\x17summary_cross_entropy_1\x15+'\x8c>"
step 1080 : accuracy: b'\n\x17\n\x10summary_accuracy\x15\xd7\xa3p?'
step 1090 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15\xa6D\x8a>'
step 1090 : accuracy: b'\n\x17\n\x10summary_accuracy\x15\x8f\xc2u?'

test_accuracy: 0.8656 

step 1100 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15\xd4eo>'
step 1100 : accuracy: b'\n\x17\n\x10summary_accuracy\x15\x1f\x85k?'

test_accuracy: 0.866 

step 1110 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15\x94\x88\x15>'
step 1110 : accuracy: b'\n\x17\n\x10summary_accuracy\x15\x00\x00\x80?'
step 1120 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15d\xac\x1f>'
step 1120 : accuracy: b'\n\x17\n\x10summary_accuracy\x15H\xe1z?'

test_accuracy: 0.8646 

step 1130 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15\xd0\t#>'
step 1130 : accuracy: b'\n\x17\n\x10summary_accuracy\x15\x00\x00\x80?'

test_accuracy:


test_accuracy: 0.875 

step 1620 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15\xec\xea\x98='
step 1620 : accuracy: b'\n\x17\n\x10summary_accuracy\x15\x00\x00\x80?'
step 1630 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15\xa5\xeaW>'
step 1630 : accuracy: b'\n\x17\n\x10summary_accuracy\x15\x8f\xc2u?'

test_accuracy: 0.8734 

step 1640 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x157\xfe%>'
step 1640 : accuracy: b'\n\x17\n\x10summary_accuracy\x15\x8f\xc2u?'

test_accuracy: 0.8732 

step 1650 : cross-entropy: b"\n\x1e\n\x17summary_cross_entropy_1\x15\n'\x92="
step 1650 : accuracy: b'\n\x17\n\x10summary_accuracy\x15\x00\x00\x80?'
step 1660 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15\x12\t\xbf='
step 1660 : accuracy: b'\n\x17\n\x10summary_accuracy\x15\x00\x00\x80?'

test_accuracy: 0.875 

step 1670 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15&\xd9\xd0='
step 1670 : accuracy: b'\n\x17\n\x10summary_accuracy\x15H\xe1z?'

test_acc


test_accuracy: 0.8718 

step 2150 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x154\xf7\xa8='
step 2150 : accuracy: b'\n\x17\n\x10summary_accuracy\x15\x00\x00\x80?'

test_accuracy: 0.8754 

step 2160 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15d\x9e\xdf='
step 2160 : accuracy: b'\n\x17\n\x10summary_accuracy\x15H\xe1z?'
step 2170 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15\x81\xc1\xef='
step 2170 : accuracy: b'\n\x17\n\x10summary_accuracy\x15\x00\x00\x80?'

test_accuracy: 0.8744 

step 2180 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15\xd2\xa4(='
step 2180 : accuracy: b'\n\x17\n\x10summary_accuracy\x15\x00\x00\x80?'

test_accuracy: 0.874 

step 2190 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15\xe0g\xb2='
step 2190 : accuracy: b'\n\x17\n\x10summary_accuracy\x15H\xe1z?'
step 2200 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15\x8d\x01\xf7='
step 2200 : accuracy: b'\n\x17\n\x10summary_accuracy\x15H\xe1z?'

test_a

...


test_accuracy: 0.8768 

step 9900 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15\xf4\xf7\x86;'
step 9900 : accuracy: b'\n\x17\n\x10summary_accuracy\x15\x00\x00\x80?'
step 9910 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15 "p;'
step 9910 : accuracy: b'\n\x17\n\x10summary_accuracy\x15\x00\x00\x80?'

test_accuracy: 0.8774 

step 9920 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15\xde\x8a\x92;'
step 9920 : accuracy: b'\n\x17\n\x10summary_accuracy\x15\x00\x00\x80?'

test_accuracy: 0.8772 

step 9930 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15WH\xb5;'
step 9930 : accuracy: b'\n\x17\n\x10summary_accuracy\x15\x00\x00\x80?'
step 9940 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15\x04\xa8\xd7;'
step 9940 : accuracy: b'\n\x17\n\x10summary_accuracy\x15\x00\x00\x80?'

test_accuracy: 0.877 

step 9950 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15\x8f3\xa6;'
step 9950 : accuracy: b'\n\x17\n\x10summary_accuracy\x15\x00\x00\x


test_accuracy: 0.8774 

step 10400 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15w\xaa\xd5;'
step 10400 : accuracy: b'\n\x17\n\x10summary_accuracy\x15\x00\x00\x80?'

test_accuracy: 0.8778 

step 10410 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15o\x8bz;'
step 10410 : accuracy: b'\n\x17\n\x10summary_accuracy\x15\x00\x00\x80?'
step 10420 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15\x9c\x0e\x06<'
step 10420 : accuracy: b'\n\x17\n\x10summary_accuracy\x15\x00\x00\x80?'

test_accuracy: 0.8778 

step 10430 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15\x85&\xdb;'
step 10430 : accuracy: b'\n\x17\n\x10summary_accuracy\x15\x00\x00\x80?'

test_accuracy: 0.8776 

step 10440 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15\xf3\x90\xa8;'
step 10440 : accuracy: b'\n\x17\n\x10summary_accuracy\x15\x00\x00\x80?'
step 10450 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15\x0f/\xf2;'
step 10450 : accuracy: b'\n\x17\n\x10summary_accura


test_accuracy: 0.8754 

step 10890 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15J\x89\xd0;'
step 10890 : accuracy: b'\n\x17\n\x10summary_accuracy\x15\x00\x00\x80?'
step 10900 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15\xeceq;'
step 10900 : accuracy: b'\n\x17\n\x10summary_accuracy\x15\x00\x00\x80?'

test_accuracy: 0.8756 

step 10910 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15\x7f2\x89;'
step 10910 : accuracy: b'\n\x17\n\x10summary_accuracy\x15\x00\x00\x80?'

test_accuracy: 0.8758 

step 10920 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15\xbd\x02\xb5;'
step 10920 : accuracy: b'\n\x17\n\x10summary_accuracy\x15\x00\x00\x80?'
step 10930 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15\x98\xfe\x8b;'
step 10930 : accuracy: b'\n\x17\n\x10summary_accuracy\x15\x00\x00\x80?'

test_accuracy: 0.876 

step 10940 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15\xf4L\x8e;'
step 10940 : accuracy: b'\n\x17\n\x10summary_accurac


test_accuracy: 0.8766 

step 11390 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15\x17D\xb3;'
step 11390 : accuracy: b'\n\x17\n\x10summary_accuracy\x15\x00\x00\x80?'

test_accuracy: 0.877 

step 11400 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15\x08\x19\x8d;'
step 11400 : accuracy: b'\n\x17\n\x10summary_accuracy\x15\x00\x00\x80?'
step 11410 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15\xfa{\xb8;'
step 11410 : accuracy: b'\n\x17\n\x10summary_accuracy\x15\x00\x00\x80?'

test_accuracy: 0.8768 

step 11420 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15\xbf\x04\x88;'
step 11420 : accuracy: b'\n\x17\n\x10summary_accuracy\x15\x00\x00\x80?'

test_accuracy: 0.8768 

step 11430 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15\xce1);'
step 11430 : accuracy: b'\n\x17\n\x10summary_accuracy\x15\x00\x00\x80?'
step 11440 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15\x8c9_;'
step 11440 : accuracy: b'\n\x17\n\x10summary_accuracy\x


test_accuracy: 0.8774 

step 11880 : cross-entropy: b"\n\x1e\n\x17summary_cross_entropy_1\x15_'L;"
step 11880 : accuracy: b'\n\x17\n\x10summary_accuracy\x15\x00\x00\x80?'
step 11890 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15o\xe9e;'
step 11890 : accuracy: b'\n\x17\n\x10summary_accuracy\x15\x00\x00\x80?'

test_accuracy: 0.8776 

step 11900 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x157\x87\x86;'
step 11900 : accuracy: b'\n\x17\n\x10summary_accuracy\x15\x00\x00\x80?'

test_accuracy: 0.8772 

step 11910 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15F@\x93;'
step 11910 : accuracy: b'\n\x17\n\x10summary_accuracy\x15\x00\x00\x80?'
step 11920 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15\xcc\x01\x9f;'
step 11920 : accuracy: b'\n\x17\n\x10summary_accuracy\x15\x00\x00\x80?'

test_accuracy: 0.8772 

step 11930 : cross-entropy: b'\n\x1e\n\x17summary_cross_entropy_1\x15\xa3\xe5\xbe;'
step 11930 : accuracy: b'\n\x17\n\x10summary_accuracy\x15\x0

![tf view elbow](https://raw.githubusercontent.com/unclebob7/ML-cheatsheet-with-TF/master/graph/tf_view_overfit.png)